In [2]:

from tensorflow.keras import layers, models

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Prepare the training data generator
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Guns_dataset/processed_dataset/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Dynamically determine the number of classes
num_classes = len(train_generator.class_indices)

# Create the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Adjust the output layer to match the number of classes
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10)

# Prepare the test data generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Guns_dataset/processed_dataset/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Verify classes
print("Training classes:", train_generator.class_indices)
print("Test classes:", test_generator.class_indices)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("\nTest Accuracy: ", test_acc)


Found 177 images belonging to 3 classes.
Epoch 1/10
6/6 [==============================] - 25s 3s/step - loss: 2.3157 - accuracy: 0.4181
Epoch 2/10
6/6 [==============================] - 12s 2s/step - loss: 0.8926 - accuracy: 0.6158
Epoch 3/10
6/6 [==============================] - 10s 2s/step - loss: 0.8192 - accuracy: 0.6328
Epoch 4/10
6/6 [==============================] - 11s 2s/step - loss: 0.6759 - accuracy: 0.6836
Epoch 5/10
6/6 [==============================] - 12s 2s/step - loss: 0.5936 - accuracy: 0.7401
Epoch 6/10
6/6 [==============================] - 11s 2s/step - loss: 0.4104 - accuracy: 0.8757
Epoch 7/10
6/6 [==============================] - 12s 2s/step - loss: 0.3362 - accuracy: 0.8757
Epoch 8/10
6/6 [==============================] - 11s 2s/step - loss: 0.2017 - accuracy: 0.9266
Epoch 9/10
6/6 [==============================] - 12s 2s/step - loss: 0.1007 - accuracy: 0.9605
Epoch 10/10
6/6 [==============================] - 11s 2s/step - loss: 0.0545 - accuracy: 0.988

In [3]:
model.save('/content/drive/MyDrive/Colab Notebooks/Guns_dataset/Restnet50_guns1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

CLASS_LABELS = ['pistol',  'revolver','rifle']
# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Guns_dataset/Restnet50_guns1.h5')

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))  # Resize image to match the input size expected by the model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to create a batch
    img_array = preprocess_input(img_array)  # Preprocess input according to the requirements of ResNet50
    return img_array

# Example usage
img_path = '/content/drive/MyDrive/Colab Notebooks/Guns_dataset/Rifles/th (33).jpeg'
img = preprocess_image(img_path)
predictions = model.predict(img)
predicted_class_index = np.argmax(predictions[0])
predicted_label = CLASS_LABELS[predicted_class_index]
print(f'The image is a : {predicted_label}')


1/1 [==============================] - 0s 128ms/step
The image is a : rifle


In [5]:
# Import necessary libraries and modules
from tensorflow.keras.applications import VGG16, MobileNetV2, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

# Prepare the training and test data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Guns_dataset/processed_dataset/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Guns_dataset/processed_dataset/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Ensure the order of the test data is consistent for evaluation
)

# Dynamically determine the number of classes
num_classes = len(train_generator.class_indices)

# Function to build VGG16 model
def build_vgg16(num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Function to build MobileNetV2 model
def build_mobilenetv2(num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense


Found 177 images belonging to 3 classes.
Found 61 images belonging to 3 classes.


In [8]:
def train_and_evaluate_model(model, train_generator, test_generator):
    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(train_generator, epochs=10)

    # Predict the labels for the test dataset
    test_generator.reset()  # Reset the generator before prediction
    predictions = model.predict(test_generator)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = test_generator.classes

    # Compute the confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    return conf_matrix, accuracy

def build_resnet50(num_classes):
    # Load the ResNet50 model pre-trained on ImageNet data
    base_model = ResNet50(weights='imagenet', include_top=False)

    # Freeze the layers in the base ResNet50 model
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Combine the base model with custom classification head
    model = Model(inputs=base_model.input, outputs=predictions)

    return model


# ResNet50
resnet_model = build_resnet50(num_classes)
resnet_conf_matrix, resnet_accuracy = train_and_evaluate_model(resnet_model, train_generator, test_generator)
print("ResNet50 Confusion Matrix:")
print(resnet_conf_matrix)
print("ResNet50 Accuracy:", resnet_accuracy)

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/10
6/6 [==============================] - 22s 3s/step - loss: 1.3414 - accuracy: 0.4915
Epoch 2/10
6/6 [==============================] - 20s 3s/step - loss: 0.9943 - accuracy: 0.6102
Epoch 3/10
6/6 [==============================] - 17s 3s/step - loss: 0.9270 - accuracy: 0.6215
Epoch 4/10
6/6 [==============================] - 17s 3s/step - loss: 0.8952 - accuracy: 0.6271
Epoch 5/10
6/6 [==============================] - 18s 3s/step - loss: 0.8456 - accuracy: 0.6271
Epoch 6/10
6/6 [==============================] - 18s 3s/step - loss: 0.8474 - accuracy: 0.6271
Epoch 7/10
6/6 [==============================] - 17s 3s/step - loss: 0.8283 - accuracy: 0.6271
Epoch 8/10
6/6 [==============================] - 17s 3s/step - loss: 0.8509 - accuracy: 0.6441
Epoch 9/10
6/6 [==============================] - 17s 3s/step - loss: 0.7864 - accuracy: 0.6610
Epoch 10/10
2/2 [==============================] - 8s 2s/step
ResNet50 